### <span style="color:#6A0DAD;"> The purpose of this notebook is to clean the datasets and pre-process them for analysis.

In [1]:
# Import libraries for data analysis, visualisation 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# Load dataframes from CSVs
transactions_data = pd.read_csv('transaction_data.csv')
demographic_data = pd.read_csv('hh_demographic.csv')
products_data = pd.read_csv('product.csv')
campaigns_data = pd.read_csv('campaign_table.csv')
camp_desc_data = pd.read_csv('campaign_desc.csv')
coupons_data = pd.read_csv('coupon.csv')
coupon_redempt_data = pd.read_csv('coupon_redempt.csv')

In [ ]:

# Print top rows of each dataframe  
print(transactions_data.head())
print(transactions_data.isnull().sum())

In [15]:
print(demographic_data.head())
print(demographic_data.isnull().sum())
print(demographic_data.info())

  AGE_DESC MARITAL_STATUS_CODE INCOME_DESC HOMEOWNER_DESC      HH_COMP_DESC  \
0      65+                   A      35-49K      Homeowner  2 Adults No Kids   
1    45-54                   A      50-74K      Homeowner  2 Adults No Kids   
2    25-34                   U      25-34K        Unknown     2 Adults Kids   
3    25-34                   U      75-99K      Homeowner     2 Adults Kids   
4    45-54                   B      50-74K      Homeowner     Single Female   

  HOUSEHOLD_SIZE_DESC KID_CATEGORY_DESC  household_key  
0                   2      None/Unknown              1  
1                   2      None/Unknown              7  
2                   3                 1              8  
3                   4                 2             13  
4                   1      None/Unknown             16  
AGE_DESC               0
MARITAL_STATUS_CODE    0
INCOME_DESC            0
HOMEOWNER_DESC         0
HH_COMP_DESC           0
HOUSEHOLD_SIZE_DESC    0
KID_CATEGORY_DESC      0
househol

In [3]:
print(products_data.head())
print(campaigns_data.head())
print(camp_desc_data.head())
print(coupons_data.head())
print(coupon_redempt_data.head())

   household_key    BASKET_ID  DAY  PRODUCT_ID  QUANTITY  SALES_VALUE  \
0           2375  26984851472    1     1004906         1         1.39   
1           2375  26984851472    1     1033142         1         0.82   
2           2375  26984851472    1     1036325         1         0.99   
3           2375  26984851472    1     1082185         1         1.21   
4           2375  26984851472    1     8160430         1         1.50   

   STORE_ID  RETAIL_DISC  TRANS_TIME  WEEK_NO  COUPON_DISC  COUPON_MATCH_DISC  
0       364        -0.60        1631        1          0.0                0.0  
1       364         0.00        1631        1          0.0                0.0  
2       364        -0.30        1631        1          0.0                0.0  
3       364         0.00        1631        1          0.0                0.0  
4       364        -0.39        1631        1          0.0                0.0  
  AGE_DESC MARITAL_STATUS_CODE INCOME_DESC HOMEOWNER_DESC      HH_COMP_DESC  \
0 

In [5]:

# Print datatype info for transactions dataframe
print(transactions_data.info())
print(demographic_data.info())
print(products_data.info())
print(campaigns_data.info())
print(camp_desc_data.info())
print(coupons_data.info())
print(coupon_redempt_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2595732 entries, 0 to 2595731
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   household_key      int64  
 1   BASKET_ID          int64  
 2   DAY                int64  
 3   PRODUCT_ID         int64  
 4   QUANTITY           int64  
 5   SALES_VALUE        float64
 6   STORE_ID           int64  
 7   RETAIL_DISC        float64
 8   TRANS_TIME         int64  
 9   WEEK_NO            int64  
 10  COUPON_DISC        float64
 11  COUPON_MATCH_DISC  float64
dtypes: float64(4), int64(8)
memory usage: 237.6 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   AGE_DESC             801 non-null    object
 1   MARITAL_STATUS_CODE  801 non-null    object
 2   INCOME_DESC          801 non-null    object
 3   HOMEOWNER_DESC       801 non

### <span style="color:#6A0DAD;"> Data Cleaning Methodology
### <span style="color:#6A0DAD;"> Step 1: Handling Missing Values
###### <span style="color:#6A0DAD;"> Removing Empty stings and null values based on their impact on the analysis

In [ ]:
empty_counts = {}

for col in products_data.columns:

    # Initialize count for each column
    count = 0

    for val in products_data[col]:

        # Check if value is string
        if isinstance(val, str):

            # Strip whitespace and check for empty
            if not val.strip():

                # Increment count
                count += 1

    # Save count for column  
    empty_counts[col] = count

# Print results  
print(empty_counts)

###### <span style="color:#6A0DAD;"> drop rows with empty strings; Since there aren't many empty rows, we can eliminate them  

In [105]:
# Columns to check for empty strings
columns_to_check = ['DEPARTMENT', 'COMMODITY_DESC']  

# Dropping rows where any of the specified columns have empty strings
products_data = products_data[~products_data[columns_to_check].apply(lambda x: x.str.strip()).eq('').any(axis=1)]
print(empty_counts)


{'PRODUCT_ID': 0, 'MANUFACTURER': 0, 'DEPARTMENT': 0, 'BRAND': 0, 'COMMODITY_DESC': 0, 'SUB_COMMODITY_DESC': 0}


###### <span style="color:#6A0DAD;"> This column doesn't have any impact on our analysis. On top of that 30% of the values in this column are missing.

In [106]:
column_to_drop = 'CURR_SIZE_OF_PRODUCT'  # Replace with your actual column name
# Dropping the column
products_data.drop(column_to_drop, axis=1, inplace=True)

###### <span style="color:#6A0DAD;"> This function counts the empty strings in every column of all the datasets.

In [107]:
def count_empty_strings(df):
    empty_counts = {}
    for col in df.columns:
        empty_counts[col] = (df[col].apply(lambda x: x.strip() == '' if isinstance(x, str) else False)).sum()
    return empty_counts

empty_counts_products = count_empty_strings(products_data)
empty_counts_transactions = count_empty_strings(transactions_data)
empty_counts_demographic = count_empty_strings(demographic_data)
empty_counts_campaigns = count_empty_strings(campaigns_data)
empty_counts_campaign_desc = count_empty_strings(camp_desc_data)
empty_counts_coupons = count_empty_strings(coupons_data)
empty_counts_redemptions = count_empty_strings(coupon_redempt_data)
# Print results
print("Empty string counts in Products Data:", empty_counts_products)
print("Empty string counts in Customers Data:", empty_counts_transactions)
print("Empty string counts in Sales Data:", empty_counts_demographic)
print("Empty string counts in Campaigns Data:", empty_counts_campaigns)
print("Empty string counts in Campaigns_desc:", empty_counts_campaign_desc)
print("Empty string counts in Coupons:", empty_counts_coupons)
print("Empty string counts in Coupons_redemtions:", empty_counts_redemptions)


Empty string counts in Products Data: {'PRODUCT_ID': 0, 'MANUFACTURER': 0, 'DEPARTMENT': 0, 'BRAND': 0, 'COMMODITY_DESC': 0, 'SUB_COMMODITY_DESC': 0}
Empty string counts in Customers Data: {'household_key': 0, 'BASKET_ID': 0, 'DAY': 0, 'PRODUCT_ID': 0, 'QUANTITY': 0, 'SALES_VALUE': 0, 'STORE_ID': 0, 'RETAIL_DISC': 0, 'TRANS_TIME': 0, 'WEEK_NO': 0, 'COUPON_DISC': 0, 'COUPON_MATCH_DISC': 0}
Empty string counts in Sales Data: {'AGE_DESC': 0, 'MARITAL_STATUS_CODE': 0, 'INCOME_DESC': 0, 'HOMEOWNER_DESC': 0, 'HH_COMP_DESC': 0, 'HOUSEHOLD_SIZE_DESC': 0, 'KID_CATEGORY_DESC': 0, 'household_key': 0}
Empty string counts in Campaigns Data: {'DESCRIPTION': 0, 'household_key': 0, 'CAMPAIGN': 0}
Empty string counts in Campaigns_desc: {'DESCRIPTION': 0, 'CAMPAIGN': 0, 'START_DAY': 0, 'END_DAY': 0}
Empty string counts in Coupons: {'COUPON_UPC': 0, 'PRODUCT_ID': 0, 'CAMPAIGN': 0}
Empty string counts in Coupons_redemtions: {'household_key': 0, 'DAY': 0, 'COUPON_UPC': 0, 'CAMPAIGN': 0}


###### <span style="color:#6A0DAD;"> This function counts the empty int values in every column of all the datasets.

In [104]:
def count_missing_numerical(df):
    missing_counts = {}
    for col in df.columns:
        # Check if the column is of a numeric datatype
        if pd.api.types.is_numeric_dtype(df[col]):
            missing_counts[col] = df[col].isna().sum()
    return missing_counts

# Apply the function to each DataFrame
missing_counts_products = count_missing_numerical(products_data)
missing_counts_customers = count_missing_numerical(transactions_data)
missing_counts_sales = count_missing_numerical(demographic_data)

# Print results
print("Missing numerical data counts in Products Data:", missing_counts_products)
print("Missing numerical data counts in Customers Data:", missing_counts_customers)
print("Missing numerical data counts in Sales Data:", missing_counts_sales)


Missing numerical data counts in Products Data: {'PRODUCT_ID': 0, 'MANUFACTURER': 0}
Missing numerical data counts in Customers Data: {'household_key': 0, 'BASKET_ID': 0, 'DAY': 0, 'PRODUCT_ID': 0, 'QUANTITY': 0, 'SALES_VALUE': 0, 'STORE_ID': 0, 'RETAIL_DISC': 0, 'TRANS_TIME': 0, 'WEEK_NO': 0, 'COUPON_DISC': 0, 'COUPON_MATCH_DISC': 0}
Missing numerical data counts in Sales Data: {'household_key': 0}


In [117]:
#Checking and counting for null
print(transactions_data.isnull().sum())
print(demographic_data.isnull().count())

print(products_data.isnull().sum())

print(campaigns_data.isnull().sum())
print(camp_desc_data.isnull().sum())
print(coupons_data.isnull().sum())
print(coupon_redempt_data.isnull().sum())


household_key                 0
BASKET_ID                     0
DAY                           0
PRODUCT_ID                    0
QUANTITY                      0
SALES_VALUE                   0
STORE_ID                      0
RETAIL_DISC                   0
TRANS_TIME                    0
WEEK_NO                       0
COUPON_DISC                   0
COUPON_MATCH_DISC             0
Loyalty_card_price        14466
Non_loyalty_card_price    14466
dtype: int64
AGE_DESC               801
MARITAL_STATUS_CODE    801
INCOME_DESC            801
HOMEOWNER_DESC         801
HH_COMP_DESC           801
HOUSEHOLD_SIZE_DESC    801
KID_CATEGORY_DESC      801
household_key          801
dtype: int64
PRODUCT_ID            0
MANUFACTURER          0
DEPARTMENT            0
BRAND                 0
COMMODITY_DESC        0
SUB_COMMODITY_DESC    0
dtype: int64
DESCRIPTION      0
household_key    0
CAMPAIGN         0
dtype: int64
DESCRIPTION    0
CAMPAIGN       0
START_DAY      0
END_DAY        0
dtype: int64
CO

In [113]:
transactions_data['Loyalty_card_price'] = (transactions_data['SALES_VALUE'] + (transactions_data['RETAIL_DISC'] + transactions_data['COUPON_DISC'])) / transactions_data['QUANTITY']
transactions_data['Non_loyalty_card_price'] = (transactions_data['SALES_VALUE'] + transactions_data['COUPON_DISC']) / transactions_data['QUANTITY']

print(transactions_data.head())
print(transactions_data.tail())

   household_key    BASKET_ID  DAY  PRODUCT_ID  QUANTITY  SALES_VALUE  \
0           2375  26984851472    1     1004906         1         1.39   
1           2375  26984851472    1     1033142         1         0.82   
2           2375  26984851472    1     1036325         1         0.99   
3           2375  26984851472    1     1082185         1         1.21   
4           2375  26984851472    1     8160430         1         1.50   

   STORE_ID  RETAIL_DISC  TRANS_TIME  WEEK_NO  COUPON_DISC  COUPON_MATCH_DISC  \
0       364        -0.60        1631        1          0.0                0.0   
1       364         0.00        1631        1          0.0                0.0   
2       364        -0.30        1631        1          0.0                0.0   
3       364         0.00        1631        1          0.0                0.0   
4       364        -0.39        1631        1          0.0                0.0   

   Loyalty_card_price  Non_loyalty_card_price  
0                0.79     

In [115]:
transactions_data['Loyalty_card_price'] = transactions_data['Loyalty_card_price'].replace([np.inf, -np.inf], np.nan)
transactions_data['Non_loyalty_card_price'] = transactions_data['Non_loyalty_card_price'].replace([np.inf, -np.inf], np.nan)
print(transactions_data.head())

   household_key    BASKET_ID  DAY  PRODUCT_ID  QUANTITY  SALES_VALUE  \
0           2375  26984851472    1     1004906         1         1.39   
1           2375  26984851472    1     1033142         1         0.82   
2           2375  26984851472    1     1036325         1         0.99   
3           2375  26984851472    1     1082185         1         1.21   
4           2375  26984851472    1     8160430         1         1.50   

   STORE_ID  RETAIL_DISC  TRANS_TIME  WEEK_NO  COUPON_DISC  COUPON_MATCH_DISC  \
0       364        -0.60        1631        1          0.0                0.0   
1       364         0.00        1631        1          0.0                0.0   
2       364        -0.30        1631        1          0.0                0.0   
3       364         0.00        1631        1          0.0                0.0   
4       364        -0.39        1631        1          0.0                0.0   

   Loyalty_card_price  Non_loyalty_card_price  
0                0.79     

###### <span style="color:#6A0DAD;">It is important to know how much each household is spending per transaction or per Basket, as we can later on build upon this to more granular analysis by demographics and product data

In [119]:
special_cases = transactions_data[transactions_data['QUANTITY'] == 0]
main_analysis = transactions_data[transactions_data['QUANTITY'] != 0]
special_cases.to_csv('special_cases.csv', index=False)
main_analysis.to_csv('main_analysis.csv', index=False)


In [120]:
transactions = transactions_data[transactions_data['QUANTITY'] != 0]
print(transactions_data.head())
print("Total rows after removing zero quantity:", len(transactions_data))


   household_key    BASKET_ID  DAY  PRODUCT_ID  QUANTITY  SALES_VALUE  \
0           2375  26984851472    1     1004906         1         1.39   
1           2375  26984851472    1     1033142         1         0.82   
2           2375  26984851472    1     1036325         1         0.99   
3           2375  26984851472    1     1082185         1         1.21   
4           2375  26984851472    1     8160430         1         1.50   

   STORE_ID  RETAIL_DISC  TRANS_TIME  WEEK_NO  COUPON_DISC  COUPON_MATCH_DISC  \
0       364        -0.60        1631        1          0.0                0.0   
1       364         0.00        1631        1          0.0                0.0   
2       364        -0.30        1631        1          0.0                0.0   
3       364         0.00        1631        1          0.0                0.0   
4       364        -0.39        1631        1          0.0                0.0   

   Loyalty_card_price  Non_loyalty_card_price  
0                0.79     

In [ ]:
Total_sales_per_basket = transactions_data.groupby('BASKET_ID')['SALES_VALUE'].sum()
print(avg_sales_per_basket)


In [ ]:
merged = demographic_data.merge(transactions_data, on='household_key') 
spend_by_income = merged.groupby('INCOME_DESC')[ 'SALES_VALUE'].sum()
print(spend_by_income)

###### <span style="color:#6A0DAD;">No of unique household_keys we have got the demographic data for. this is important to find out the most valuable customers

In [ ]:
household_counts = merged.groupby('INCOME_DESC')['household_key'].nunique()
print(household_counts)


In [ ]:
household_counts.plot(kind='bar')
plt.title('Households by Income Description')
plt.xlabel('Income Description')
plt.ylabel('Household Count')
plt.show()
#This gives us a bar graph that displays the household_count by income description

In [ ]:
# Count transactions by income Description
txns_by_income = merged.groupby('INCOME_DESC')['household_key'].count() 

# Plot bar chart
txns_by_income.plot(kind='bar')
plt.title('Transactions by Income Description')
plt.xlabel('Income Description') 
plt.ylabel('Number of Transactions')
plt.show()


###### <span style="color:#6A0DAD;">From the above visual, we can see that three income categories accounted for more than half the sales. This insight is important to look deeper into the product info on which categories were purchased by these customers the most. This digging gives insights to category and insights team  

In [ ]:
merged = transactions_data.merge(products_data, on='PRODUCT_ID')


In [ ]:
sales_by_category = merged.groupby('COMMODITY_DESC')['SALES_VALUE'].sum()


In [ ]:
top5 = (merged
   .groupby('DEPARTMENT')['SALES_VALUE']
   .sum()
   .sort_values(ascending=False)
   .head(10)
)


In [ ]:
print(top5)